In [4]:
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

IND_INIT_SIZE = 5
MAX_ITEM = 50
MAX_WEIGHT = 50
NBR_ITEMS = 20

In [35]:
# To assure reproductibility, the RNG seed is set prior to the items
# dict initialization. It is also seeded in main().
random.seed(64)

# Create the item dictionary: item name is an integer, and value is 
# a (weight, value) 2-uple.
items = {}
# Create random items and store them in the items' dictionary.
for i in range(NBR_ITEMS):
    items[i] = (random.randint(1, 10), random.uniform(0, 100))
    
items # item id: (重さ, 価値) ref => https://ja.wikipedia.org/wiki/%E3%83%8A%E3%83%83%E3%83%97%E3%82%B5%E3%83%83%E3%82%AF%E5%95%8F%E9%A1%8C

{0: (8, 12.48509930835593),
 1: (10, 40.396096956847295),
 2: (1, 20.351113369770758),
 3: (5, 70.41144439033413),
 4: (4, 69.86574666158654),
 5: (3, 7.981206373372229),
 6: (7, 44.003630577221465),
 7: (1, 8.366591038506865),
 8: (1, 92.7646786450061),
 9: (3, 10.541265149933377),
 10: (1, 95.00359117156347),
 11: (5, 21.924990817330915),
 12: (4, 8.882167797500228),
 13: (1, 76.96797658382238),
 14: (7, 89.16384444665715),
 15: (5, 74.23923323846317),
 16: (9, 42.688343941286256),
 17: (9, 30.701913496381728),
 18: (6, 68.83604339265639),
 19: (9, 17.89634902035938)}

In [46]:
creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", set, fitness=creator.Fitness)

# indは集合なので、ナップザックに入れるアイテムは1種類1個だけ。

/home/yuki/anaconda3/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/yuki/anaconda3/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [2]:
set1 = set([1,2,2,3])
set1

{1, 2, 3}

In [37]:
toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_item", random.randrange, NBR_ITEMS)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [38]:
def evalKnapsack(individual):
    weight = 0.0
    value = 0.0
    for item in individual:
        weight += items[item][0]
        value += items[item][1]
    if len(individual) > MAX_ITEM or weight > MAX_WEIGHT:
        return 10000, 0             # Ensure overweighted bags are dominated
    return weight, value

def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)                # Used in order to keep type
    ind1 &= ind2                    # Intersection (inplace)
    ind2 ^= temp                    # Symmetric Difference (inplace)
    return ind1, ind2
    
def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:     # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,

toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

In [39]:
7 & 5 # bit and

5

In [40]:
10 ^ 12 # bit xor

6

In [41]:
def main():
    random.seed(64)
    NGEN = 50
    MU = 50
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2
    
    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)
    
    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)
    
    return pop, stats, hof

In [42]:
pop, stats, hof = main()

gen	nevals	avg                          	std                        	min                        	max                          
0  	50    	[  22.78        210.00877715]	[  5.88316241  71.09309649]	[ 10.          49.69958685]	[  38.          345.35491309]
1  	87    	[   9.96        145.20790666]	[  10.61312395  139.1868852 ]	[ 0.  0.]                  	[  45.          414.48478381]
2  	91    	[  3.26        61.20087478]  	[   7.44797959  125.53892809]	[ 0.  0.]                  	[  28.          414.48478381]
3  	88    	[  4.6         84.51641114]  	[   8.57438044  140.51459866]	[ 0.  0.]                  	[  28.          414.48478381]
4  	92    	[  2.4         52.24310488]  	[   5.82065288  108.88598622]	[ 0.  0.]                  	[  28.          414.48478381]
5  	87    	[  3.66        74.97342258]  	[   6.99316809  127.02866009]	[ 0.  0.]                  	[  28.          414.48478381]
6  	82    	[   5.3         111.43072783]	[   7.61117599  142.76899122]	[ 0.  0.]                  	[ 

In [43]:
pop

[{3, 4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
 {3, 4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
 set(),
 {2, 4, 7, 8, 10, 13, 15},
 {2, 4, 8, 10, 13, 14, 15, 18},
 {4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
 {4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
 {4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
 {4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
 {4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
 {4, 7, 8, 10, 12, 13, 14, 15, 16, 18},
 {3, 4, 6, 8, 9, 10, 13, 14, 15},
 {4, 7, 8, 9, 10, 12, 13, 14, 15, 18},
 set(),
 {10},
 {10},
 {2, 8, 10, 13},
 {4, 8, 10, 13},
 {8, 10},
 {8, 10, 13},
 {2, 4, 7, 8, 10, 13},
 {2, 4, 7, 8, 10, 13},
 {2, 4, 7, 8, 10, 13},
 {2, 4, 7, 8, 10, 13},
 {4, 8, 10, 13, 15},
 {4, 8, 10, 13, 14, 15, 18},
 {4, 7, 8, 9, 10, 13, 14, 15},
 {4, 7, 8, 9, 10, 13, 14, 15},
 {4, 7, 8, 9, 10, 13, 14, 15},
 {2, 4, 8, 10, 13, 15},
 {4, 7, 8, 9, 10, 12, 13, 14, 15, 18},
 {2, 8, 10, 13},
 {8, 10, 13},
 {4, 8, 10, 13},
 {4, 8, 10, 13, 15},
 {4, 8, 10, 13, 14, 15, 18},
 {2, 4, 8, 10, 13},
 {4, 7, 8, 10, 13

In [44]:
hof.items

[set(),
 {10},
 {8, 10},
 {8, 10, 13},
 {2, 8, 10, 13},
 {4, 8, 10, 13},
 {2, 4, 8, 10, 13},
 {2, 4, 7, 8, 10, 13},
 {4, 8, 10, 13, 15},
 {2, 4, 8, 10, 13, 15},
 {2, 4, 7, 8, 10, 13, 15},
 {4, 8, 10, 13, 14, 15},
 {4, 7, 8, 10, 13, 14, 15},
 {4, 8, 9, 10, 13, 14, 15},
 {4, 7, 8, 9, 10, 13, 14, 15},
 {4, 8, 10, 13, 14, 15, 18},
 {2, 4, 8, 10, 13, 14, 15, 18},
 {4, 7, 8, 9, 10, 12, 13, 14, 15, 18},
 {3, 4, 6, 8, 9, 10, 13, 14, 15},
 {4, 7, 8, 10, 12, 13, 14, 15, 16, 18},
 {4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
 {3, 4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18}]

In [45]:
stats.fields

['avg', 'std', 'min', 'max']